# Boss Mobbing

## Description

When agents want to mob their boss by repeatedly performing poorly as a team.

In [677]:
import pandas as pd
import numpy as np
import altair as alt
import networkx as nx
alt.data_transformers.enable('vegafusion')

DataTransformerRegistry.enable('vegafusion')

In [678]:
log_name = 'propr_test'

fn_logs = {
    'bpic15': '../data/preproc/bpic15.csv',
    'bpic15_ref': '../data/preproc/bpic15_train-70.csv',
    'bpic15_test': '../data/preproc/bpic15_test-30.csv',

    'bpic17': '../data/preproc/bpic2017.csv',

    'propr': '../data/preproc/proprietary.csv',
    'propr_ref': '../data/preproc/proprietary_train-70.csv',
    'propr_test': '../data/preproc/proprietary_test-30.csv'
}

In [679]:
log = pd.read_csv(fn_logs[log_name])
log

,activity,case_id,start timestamp,complete timestamp,resource,Weekday,Cost,"School holidays; 0 = no, 1 = yes",Approval; 1 = low risk to 4 = high risk,Type of loan; 0 = normal; 1 = special,_start timestamp,_complete timestamp
0,Application_Processing_Branches,ID-799,25/02/2011 16:01,2011-02-25 16:10:00,070-6-05,Friday,"1,02 € per minute",0.0,1.0,0.0,2011-02-25 16:01:00,2011-02-25 16:10:00
1,Precheck,ID-940,25/02/2011 16:06,2011-02-25 16:19:00,070-9-2,Friday,"1,02 € per minute",0.0,1.0,0.0,2011-02-25 16:06:00,2011-02-25 16:19:00
2,Precheck,ID-1078,28/02/2011 8:28,2011-02-28 09:02:00,001-6-02,Monday,"0,87 € per minute",0.0,1.0,0.0,2011-02-28 08:28:00,2011-02-28 09:02:00
3,Precheck,ID-546,28/02/2011 10:40,2011-02-28 10:45:00,080-10-03,Monday,"0,87 € per minute",0.0,1.0,0.0,2011-02-28 10:40:00,2011-02-28 10:45:00
4,Application_Processing_Branches,ID-1457,28/02/2011 10:10,2011-02-28 10:55:00,070-7-06,Monday,"1,02 € per minute",0.0,1.0,0.0,2011-02-28 10:10:00,2011-02-28 10:55:00
...,...,...,...,...,...,...,...,...,...,...,...,...
5285,Approval_Branch,ID-496,31/05/2011 14:41,2011-05-31 16:11:00,001-7-03,Tuesday,"0,87 € per minute",0.0,2.0,0.0,2011-05-31 14:41:00,2011-05-31 16:11:00
5286,Precheck,ID-1384,31/05/2011 16:20,2011-05-31 16:21:00,010-21-01,Tuesday,"0,87 € per minute",0.0,2.0,0.0,2011-05-31 16:20:00,2011-05-31 16:21:00
5287,Approval_Branch,ID-496,31/05/2011 16:29,2011-05-31 16:30:00,010-21-01,Tuesday,"0,87 € per minute",0.0,2.0,0.0,2011-05-31 16:29:00,2011-05-31 16:30:00
5288,Check_of_Documents,ID-99,31/05/2011 15:42,2011-05-31 16:30:00,010-23-06,Tuesday,"1,02 € per minute",0.0,2.0,0.0,2011-05-31 15:42:00,2011-05-31 16:30:00


In [680]:
# proprietary
log['department'] = log['resource'].apply(
    lambda x: x.split('-')[0]
)
log['department_role'] = log['resource'].apply(
    lambda x: x.split('-')[0] + '-' + x.split('-')[1]
)
log

,activity,case_id,start timestamp,complete timestamp,resource,Weekday,Cost,"School holidays; 0 = no, 1 = yes",Approval; 1 = low risk to 4 = high risk,Type of loan; 0 = normal; 1 = special,_start timestamp,_complete timestamp,department,department_role
0,Application_Processing_Branches,ID-799,25/02/2011 16:01,2011-02-25 16:10:00,070-6-05,Friday,"1,02 € per minute",0.0,1.0,0.0,2011-02-25 16:01:00,2011-02-25 16:10:00,070,070-6
1,Precheck,ID-940,25/02/2011 16:06,2011-02-25 16:19:00,070-9-2,Friday,"1,02 € per minute",0.0,1.0,0.0,2011-02-25 16:06:00,2011-02-25 16:19:00,070,070-9
2,Precheck,ID-1078,28/02/2011 8:28,2011-02-28 09:02:00,001-6-02,Monday,"0,87 € per minute",0.0,1.0,0.0,2011-02-28 08:28:00,2011-02-28 09:02:00,001,001-6
3,Precheck,ID-546,28/02/2011 10:40,2011-02-28 10:45:00,080-10-03,Monday,"0,87 € per minute",0.0,1.0,0.0,2011-02-28 10:40:00,2011-02-28 10:45:00,080,080-10
4,Application_Processing_Branches,ID-1457,28/02/2011 10:10,2011-02-28 10:55:00,070-7-06,Monday,"1,02 € per minute",0.0,1.0,0.0,2011-02-28 10:10:00,2011-02-28 10:55:00,070,070-7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5285,Approval_Branch,ID-496,31/05/2011 14:41,2011-05-31 16:11:00,001-7-03,Tuesday,"0,87 € per minute",0.0,2.0,0.0,2011-05-31 14:41:00,2011-05-31 16:11:00,001,001-7
5286,Precheck,ID-1384,31/05/2011 16:20,2011-05-31 16:21:00,010-21-01,Tuesday,"0,87 € per minute",0.0,2.0,0.0,2011-05-31 16:20:00,2011-05-31 16:21:00,010,010-21
5287,Approval_Branch,ID-496,31/05/2011 16:29,2011-05-31 16:30:00,010-21-01,Tuesday,"0,87 € per minute",0.0,2.0,0.0,2011-05-31 16:29:00,2011-05-31 16:30:00,010,010-21
5288,Check_of_Documents,ID-99,31/05/2011 15:42,2011-05-31 16:30:00,010-23-06,Tuesday,"1,02 € per minute",0.0,2.0,0.0,2011-05-31 15:42:00,2011-05-31 16:30:00,010,010-23


## Social-SB-1

In [681]:
# convert to timestamps to calculate case/activity duration

log = log.sort_values(by='start timestamp')
log['start timestamp'] = pd.to_datetime(log['start timestamp'], format='mixed', dayfirst=True)
log['complete timestamp'] = pd.to_datetime(log['complete timestamp'], format='mixed', dayfirst=True)
log['activity_duration_minutes'] = (
    log['complete timestamp'] - log['start timestamp']
).dt.total_seconds() / 60

### Construct resource handover-of-work (HOW) network

In [682]:
all_resources = sorted(log['resource'].unique())
how = np.zeros((len(all_resources), len(all_resources)))
log = log.sort_values(by='complete timestamp')
for case_id, events in log.groupby('case_id'):
    res_seq = list(events['resource'])
    for i, r in enumerate(res_seq):
        px = all_resources.index(r)
        # ignore if the last resource in case
        if i + 1 == len(res_seq):
            pass
        else:
            r_next = res_seq[i+1]
            py = all_resources.index(r_next)
            how[px,py] += 1
how = pd.DataFrame.from_dict(how)
how['resource'] = all_resources
how = how.set_index('resource')
how = how.rename(columns=dict(
    (k, all_resources[k])
    for k in range(len(all_resources))
))
print(how)
print(sum((how > 0).sum()))
how = nx.DiGraph(how)
print(how)
nx.write_graphml(how, fn_logs[log_name] + '.how.graphml')

          000-0  000-1-01  000-2-01  000-3-01  000-4-01  000-4-02  001-10-01  \
resource                                                                       
000-0       0.0       0.0       0.0       0.0       0.0       0.0        0.0   
000-1-01    0.0       0.0       0.0       0.0       0.0       0.0        0.0   
000-2-01    0.0       0.0      29.0       0.0       0.0       0.0        0.0   
000-3-01    0.0       1.0      23.0      10.0       0.0       0.0        0.0   
000-4-01    0.0       0.0       2.0       0.0       3.0       0.0        0.0   
...         ...       ...       ...       ...       ...       ...        ...   
110-8-2     0.0       0.0       0.0       3.0       0.0       0.0        0.0   
110-8-3     0.0       0.0       0.0       4.0       0.0       0.0        0.0   
111-8-2     0.0       0.0       0.0       2.0       0.0       0.0        0.0   
111-9-1     0.0       0.0       0.0       1.0       0.0       0.0        0.0   
33-01       0.0       0.0       0.0     

### Apply community detection to determine resource groups (communities)

In [683]:
# communities = nx.community.louvain_communities(how, seed=0)

In [684]:
communities = [
    # list(log.loc[log['department'] == g, 'resource'].unique())
    # for g in log['department'].unique()
    list(log.loc[log['department'] == '010', 'resource'].unique())
]

### Determine the identities of "boss" resources in each resource group

In [685]:
sel_rgs = dict()
sel_rgs_boss = dict()
for ic, c in enumerate(communities):
    # consider only group with more than 3 members
    # consider group where the resource with the highest centrality is
    # significantly higher than all others
    print('Community {}: '.format(ic))
    if len(c) >= 3:
        print('{} resources'.format(len(c)))
        sub_how = nx.subgraph(how, c)
        nx.write_graphml(sub_how, fn_logs[log_name] + f'.how_c{ic}.graphml')
        print(c)
        centrality_stats = pd.DataFrame.from_dict(
            # Coordinator: measured using betweenness centrality
            # nx.eigenvector_centrality(sub_how, max_iter=10000), 
            nx.betweenness_centrality(sub_how), 
            orient='index'
        ).reset_index().rename(columns={
            'index': 'resource',
            0: 'measured_value'
        }).sort_values(
            by='measured_value', ascending=False
        )
        ub = (
            centrality_stats['measured_value'].quantile(0.75) +
            1.5 * (centrality_stats['measured_value'].quantile(0.75) -
            centrality_stats['measured_value'].quantile(0.25))
        )
        if centrality_stats.iloc[0]['measured_value'] > ub:
            print(ub)
            print(centrality_stats)
            sel_rgs[ic] = c
            sel_rgs_boss[ic] = centrality_stats.iloc[0]['resource']
print(len(sel_rgs))
print(sel_rgs_boss)

Community 0: 
43 resources
['010-6-05', '010-9-1', '010-8-1', '010-23-13', '010-24-13', '010-21-01', '010-24-26', '010-24-02', '010-7-04', '010-23-07', '010-29-01', '010-7-01', '010-30-01', '010-23-02', '010-25-06', '010-24-11', '010-23-11', '010-24-04', '010-24-23', '010-24-17', '010-6-02', '010-25-01', '010-5-02', '010-24-12', '010-8-5', '010-24-01', '010-24-16', '010-23-15', '010-25-07', '010-23-05', '010-23-06', '010-23-01', '010-7-03', '010-23-04', '010-10-01', '010-11-02', '010-25-02', '010-23-08', '010-22-1', '010-8-4', '010-5-01', '010-10-03', '010-24-03']
0.08819768799080646
     resource  measured_value
37  010-24-26        0.463003
24  010-23-13        0.327827
34  010-24-23        0.063485
15  010-25-07        0.059748
5   010-10-01        0.045296
39  010-24-12        0.045296
18   010-7-01        0.044924
38    010-8-1        0.043388
7   010-23-02        0.043353
17  010-24-17        0.036786
42  010-24-13        0.035339
14   010-6-02        0.035219
28  010-24-04      

### Discover co-presence of resource-boss

- Direct co-presence: resources worked in the same case
- Indirect co-presence: working time of resources overlap

In [686]:
res_attendance = dict()
for res, events in log.groupby('resource'):
    res_attendance[res] = [
        pd.Interval(e['start timestamp'], e['complete timestamp'], closed='both')
        for i, e in events.iterrows()
    ]

In [687]:
# Calculate direct co-presence
if True:
    direct_coapp = []
    for case, events in log.groupby('case_id'):
        case_team = set(events['resource'].unique())
        for i, e in events.iterrows():
            direct_coapp.append((
                i, 
                ','.join(sorted(case_team.difference({e['resource']})))
            ))
    direct_coapp = pd.DataFrame(
        direct_coapp, columns=['index', 'direct_copresenting_resources']
    )
    direct_coapp = direct_coapp.set_index(keys='index', drop=True)
    log = pd.merge(log, direct_coapp, left_index=True, right_index=True)

In [688]:
# Calculate indirect co-presence:
if True:
    all_resources = set(log['resource'].unique())
    indirect_coapp = []
    for res, events in log.groupby('resource'):
        for i, e in events.iterrows():
            overlapped_resources = set()
            interval = pd.Interval(
                e['start timestamp'], e['complete timestamp'], closed='both'
            )
            for other in all_resources.difference({res}):
                for x in res_attendance[other]:
                    if interval.overlaps(x):
                        overlapped_resources.add(other)
                        break
            indirect_coapp.append((i, ','.join(sorted(overlapped_resources))))
    indirect_coapp = pd.DataFrame(
        indirect_coapp, columns=['index', 'indirect_copresenting_resources']
    )
    indirect_coapp = indirect_coapp.set_index(keys='index', drop=True)
    log = pd.merge(log, indirect_coapp, left_index=True, right_index=True)

In [689]:
# log.to_csv(fn_logs[log_name] + '.coapp.csv', index=False)
# log = pd.read_csv(fn_logs[log_name] + '.coapp.csv')
# log

In [690]:
# label events by resource groups (communities) and boss resources
for ic, c in sel_rgs.items():
    log.loc[log['resource'].isin(c), 'resource_group'] = ic
    log.loc[log['resource'].isin(c), 'resource_group_boss'] = sel_rgs_boss[ic]
log

,activity,case_id,start timestamp,complete timestamp,resource,Weekday,Cost,"School holidays; 0 = no, 1 = yes",Approval; 1 = low risk to 4 = high risk,Type of loan; 0 = normal; 1 = special,_start timestamp,_complete timestamp,department,department_role,activity_duration_minutes,direct_copresenting_resources,indirect_copresenting_resources,resource_group,resource_group_boss
0,Application_Processing_Branches,ID-799,2011-02-25 16:01:00,2011-02-25 16:10:00,070-6-05,Friday,"1,02 € per minute",0.0,1.0,0.0,2011-02-25 16:01:00,2011-02-25 16:10:00,070,070-6,9.0,,070-9-2,NaN,NaN
1,Precheck,ID-940,2011-02-25 16:06:00,2011-02-25 16:19:00,070-9-2,Friday,"1,02 € per minute",0.0,1.0,0.0,2011-02-25 16:06:00,2011-02-25 16:19:00,070,070-9,13.0,"000-2-01,010-23-02,010-23-13,010-24-13",070-6-05,NaN,NaN
2,Precheck,ID-1078,2011-02-28 08:28:00,2011-02-28 09:02:00,001-6-02,Monday,"0,87 € per minute",0.0,1.0,0.0,2011-02-28 08:28:00,2011-02-28 09:02:00,001,001-6,34.0,"010-24-23,010-24-26",,NaN,NaN
3,Precheck,ID-546,2011-02-28 10:40:00,2011-02-28 10:45:00,080-10-03,Monday,"0,87 € per minute",0.0,1.0,0.0,2011-02-28 10:40:00,2011-02-28 10:45:00,080,080-10,5.0,010-23-13,"010-6-05,020-11-01,070-7-06",NaN,NaN
4,Application_Processing_Branches,ID-1457,2011-02-28 10:10:00,2011-02-28 10:55:00,070-7-06,Monday,"1,02 € per minute",0.0,1.0,0.0,2011-02-28 10:10:00,2011-02-28 10:55:00,070,070-7,45.0,,"010-6-05,020-11-01,080-10-03",NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5285,Approval_Branch,ID-496,2011-05-31 14:41:00,2011-05-31 16:11:00,001-7-03,Tuesday,"0,87 € per minute",0.0,2.0,0.0,2011-05-31 14:41:00,2011-05-31 16:11:00,001,001-7,90.0,010-21-01,"001-8-04,001-9-01,010-23-06,010-24-17,010-24-2...",NaN,NaN
5286,Precheck,ID-1384,2011-05-31 16:20:00,2011-05-31 16:21:00,010-21-01,Tuesday,"0,87 € per minute",0.0,2.0,0.0,2011-05-31 16:20:00,2011-05-31 16:21:00,010,010-21,1.0,001-6-01,010-23-06,0.0,010-24-26
5287,Approval_Branch,ID-496,2011-05-31 16:29:00,2011-05-31 16:30:00,010-21-01,Tuesday,"0,87 € per minute",0.0,2.0,0.0,2011-05-31 16:29:00,2011-05-31 16:30:00,010,010-21,1.0,001-7-03,010-23-06,0.0,010-24-26
5288,Check_of_Documents,ID-99,2011-05-31 15:42:00,2011-05-31 16:30:00,010-23-06,Tuesday,"1,02 € per minute",0.0,2.0,0.0,2011-05-31 15:42:00,2011-05-31 16:30:00,010,010-23,48.0,"000-4-02,010-23-13,100-10-02","001-7-03,001-8-04,001-9-01,010-21-01,010-25-01",0.0,010-24-26


In [691]:
log = log.dropna(how='any')

In [692]:
# label events by direct/indirect co-presence with boss
def is_resource_in(r, str_resources):
    resources = str_resources.split(',')
    if len(resources) > 0:
        return r in resources
    return np.nan

log['direct_copresenting_boss'] = log.apply(
    lambda row: is_resource_in(
        row['resource_group_boss'], row['direct_copresenting_resources']
    ),
    axis=1
)
log['indirect_copresenting_boss'] = log.apply(
    lambda row: is_resource_in(
        row['resource_group_boss'], row['indirect_copresenting_resources']
    ),
    axis=1
)
log

C:\Users\yangj29\AppData\Local\Temp\ipykernel_14988\621649649.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  log['direct_copresenting_boss'] = log.apply(
C:\Users\yangj29\AppData\Local\Temp\ipykernel_14988\621649649.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  log['indirect_copresenting_boss'] = log.apply(


,activity,case_id,start timestamp,complete timestamp,resource,Weekday,Cost,"School holidays; 0 = no, 1 = yes",Approval; 1 = low risk to 4 = high risk,Type of loan; 0 = normal; 1 = special,...,_complete timestamp,department,department_role,activity_duration_minutes,direct_copresenting_resources,indirect_copresenting_resources,resource_group,resource_group_boss,direct_copresenting_boss,indirect_copresenting_boss
5,Application_Processing_Branches,ID-554,2011-02-28 09:38:00,2011-02-28 11:34:00,010-6-05,Monday,"0,87 € per minute",0.0,2.0,0.0,...,2011-02-28 11:34:00,010,010-6,116.0,"010-21-01,010-24-02,010-24-26,010-6-02","020-11-01,070-7-06,080-10-03",0.0,010-24-26,True,False
7,Precheck,ID-315,2011-02-28 12:17:00,2011-02-28 12:22:00,010-9-1,Monday,"0,87 € per minute",0.0,1.0,0.0,...,2011-02-28 12:22:00,010,010-9,5.0,"010-24-04,010-24-26","010-6-05,092-9-1",0.0,010-24-26,True,False
9,Approval_Branch,ID-554,2011-02-28 11:39:00,2011-02-28 14:28:00,010-6-05,Monday,"0,87 € per minute",0.0,2.0,0.0,...,2011-02-28 14:28:00,010,010-6,169.0,"010-21-01,010-24-02,010-24-26,010-6-02","010-8-1,010-9-1,020-11-01,070-7-01,092-9-1",0.0,010-24-26,True,False
11,Approval_Branch,ID-427,2011-02-28 13:08:00,2011-02-28 14:49:00,010-8-1,Monday,"0,87 € per minute",0.0,2.0,0.0,...,2011-02-28 14:49:00,010,010-8,101.0,"010-23-07,010-23-13,010-9-1","010-6-05,070-7-01,092-9-1",0.0,010-24-26,False,False
16,Application_Processing_Branches,ID-940,2011-02-28 17:37:00,2011-02-28 17:41:00,010-23-13,Monday,"1,02 € per minute",0.0,1.0,0.0,...,2011-02-28 17:41:00,010,010-23,4.0,"000-2-01,010-23-02,010-24-13,070-9-2",070-15-01,0.0,010-24-26,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5282,Processing_Approval,ID-578,2011-05-31 15:46:00,2011-05-31 15:51:00,010-25-01,Tuesday,"1,02 € per minute",0.0,3.0,0.0,...,2011-05-31 15:51:00,010,010-25,5.0,"000-2-01,000-3-01,001-8-04,010-23-13","001-7-03,001-8-04,001-9-01,010-23-06",0.0,010-24-26,False,False
5283,Refusal_Branches,ID-578,2011-05-31 15:51:00,2011-05-31 15:53:00,010-25-01,Tuesday,"1,02 € per minute",0.0,3.0,0.0,...,2011-05-31 15:53:00,010,010-25,2.0,"000-2-01,000-3-01,001-8-04,010-23-13","001-7-03,001-8-04,010-23-06",0.0,010-24-26,False,False
5286,Precheck,ID-1384,2011-05-31 16:20:00,2011-05-31 16:21:00,010-21-01,Tuesday,"0,87 € per minute",0.0,2.0,0.0,...,2011-05-31 16:21:00,010,010-21,1.0,001-6-01,010-23-06,0.0,010-24-26,False,False
5287,Approval_Branch,ID-496,2011-05-31 16:29:00,2011-05-31 16:30:00,010-21-01,Tuesday,"0,87 € per minute",0.0,2.0,0.0,...,2011-05-31 16:30:00,010,010-21,1.0,001-7-03,010-23-06,0.0,010-24-26,False,False


### Profile resource productivity

In [693]:
log['productivity'] = 1 / log['activity_duration_minutes']
log

C:\Users\yangj29\AppData\Local\Temp\ipykernel_14988\91288244.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  log['productivity'] = 1 / log['activity_duration_minutes']


,activity,case_id,start timestamp,complete timestamp,resource,Weekday,Cost,"School holidays; 0 = no, 1 = yes",Approval; 1 = low risk to 4 = high risk,Type of loan; 0 = normal; 1 = special,...,department,department_role,activity_duration_minutes,direct_copresenting_resources,indirect_copresenting_resources,resource_group,resource_group_boss,direct_copresenting_boss,indirect_copresenting_boss,productivity
5,Application_Processing_Branches,ID-554,2011-02-28 09:38:00,2011-02-28 11:34:00,010-6-05,Monday,"0,87 € per minute",0.0,2.0,0.0,...,010,010-6,116.0,"010-21-01,010-24-02,010-24-26,010-6-02","020-11-01,070-7-06,080-10-03",0.0,010-24-26,True,False,0.008621
7,Precheck,ID-315,2011-02-28 12:17:00,2011-02-28 12:22:00,010-9-1,Monday,"0,87 € per minute",0.0,1.0,0.0,...,010,010-9,5.0,"010-24-04,010-24-26","010-6-05,092-9-1",0.0,010-24-26,True,False,0.200000
9,Approval_Branch,ID-554,2011-02-28 11:39:00,2011-02-28 14:28:00,010-6-05,Monday,"0,87 € per minute",0.0,2.0,0.0,...,010,010-6,169.0,"010-21-01,010-24-02,010-24-26,010-6-02","010-8-1,010-9-1,020-11-01,070-7-01,092-9-1",0.0,010-24-26,True,False,0.005917
11,Approval_Branch,ID-427,2011-02-28 13:08:00,2011-02-28 14:49:00,010-8-1,Monday,"0,87 € per minute",0.0,2.0,0.0,...,010,010-8,101.0,"010-23-07,010-23-13,010-9-1","010-6-05,070-7-01,092-9-1",0.0,010-24-26,False,False,0.009901
16,Application_Processing_Branches,ID-940,2011-02-28 17:37:00,2011-02-28 17:41:00,010-23-13,Monday,"1,02 € per minute",0.0,1.0,0.0,...,010,010-23,4.0,"000-2-01,010-23-02,010-24-13,070-9-2",070-15-01,0.0,010-24-26,False,False,0.250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5282,Processing_Approval,ID-578,2011-05-31 15:46:00,2011-05-31 15:51:00,010-25-01,Tuesday,"1,02 € per minute",0.0,3.0,0.0,...,010,010-25,5.0,"000-2-01,000-3-01,001-8-04,010-23-13","001-7-03,001-8-04,001-9-01,010-23-06",0.0,010-24-26,False,False,0.200000
5283,Refusal_Branches,ID-578,2011-05-31 15:51:00,2011-05-31 15:53:00,010-25-01,Tuesday,"1,02 € per minute",0.0,3.0,0.0,...,010,010-25,2.0,"000-2-01,000-3-01,001-8-04,010-23-13","001-7-03,001-8-04,010-23-06",0.0,010-24-26,False,False,0.500000
5286,Precheck,ID-1384,2011-05-31 16:20:00,2011-05-31 16:21:00,010-21-01,Tuesday,"0,87 € per minute",0.0,2.0,0.0,...,010,010-21,1.0,001-6-01,010-23-06,0.0,010-24-26,False,False,1.000000
5287,Approval_Branch,ID-496,2011-05-31 16:29:00,2011-05-31 16:30:00,010-21-01,Tuesday,"0,87 € per minute",0.0,2.0,0.0,...,010,010-21,1.0,001-7-03,010-23-06,0.0,010-24-26,False,False,1.000000


### Find correlation between co-presence periods and changed performance of resources

In [694]:
col_copresence = 'direct_copresenting_boss'
res_copresence_time = log[
    ['resource', 'activity', 'activity_duration_minutes', col_copresence]
].groupby(['resource', col_copresence]).agg(
    total_act_dur=pd.NamedAgg('activity_duration_minutes', aggfunc='sum'),
    mean_act_dur=pd.NamedAgg('activity_duration_minutes', aggfunc='mean')
).reset_index()

In [695]:
res_worktime = res_copresence_time.pivot(
    index='resource', 
    columns=col_copresence, values=['total_act_dur']
).reset_index().rename(columns={
    # NOTE: level-1 columns from the multiindex columns
    '': 'resource',
    False: 'total_act_dur_wo_boss',
    True: 'total_act_dur_w_boss',
})
res_worktime.columns = res_worktime.columns.get_level_values(1)

# filter out resources without comparison possibility
res_worktime = res_worktime.dropna()
res_worktime['worktime_diff'] = res_worktime['total_act_dur_w_boss'] - res_worktime['total_act_dur_wo_boss']

In [696]:
res_productivity = res_copresence_time.pivot(
    index='resource', 
    columns=col_copresence, values=['mean_act_dur']
).reset_index().rename(columns={
    # NOTE: level-1 columns from the multiindex columns
    '': 'resource',
    False: 'mean_act_dur_wo_boss',
    True: 'mean_act_dur_w_boss',
})
res_productivity.columns = res_productivity.columns.get_level_values(1)

# filter out resources without comparison possibility
res_productivity = res_productivity.dropna()
res_productivity['productivity_wo_boss'] = 1 / res_productivity['mean_act_dur_wo_boss']
res_productivity['productivity_w_boss'] = 1 / res_productivity['mean_act_dur_w_boss']
res_productivity['productivity_diff'] = res_productivity['productivity_w_boss'] - res_productivity['productivity_wo_boss']

In [697]:
alt.hconcat(
    # chart showing difference of working time w/wo boss
    # alt.Chart(res_worktime).mark_bar().encode(
    #     x=alt.X('resource:N').title('resource ID'),
    #     y=alt.Y('worktime_diff:Q').title(['difference between work time', 'with boss and without boss']),
    # ).properties(height=220, width=270),
    # chart showing difference of productivity w/wo boss
    alt.layer(
        alt.Chart(res_productivity).mark_bar().encode(
            x=alt.X('resource:N').sort('-y').title('resource ID'),
            y=alt.Y('productivity_diff:Q').title(['difference between productivity', 'with boss and without boss']),
        ),
        alt.Chart(res_productivity).mark_text(
            align='center', angle=270, baseline='alphabetic', dx=20, dy=5
        ).encode(
            x=alt.X('resource:N').sort('-y'),
            y=alt.Y('productivity_diff:Q'),
            text=alt.Text('productivity_diff:Q', format='+.0%'),
            color=alt.condition(
                alt.datum.productivity_diff > 0,
                alt.value('black'),
                alt.value('darkorange')
            )
        )
    ).properties(height=250, width=300)
)

alt.HConcatChart(...)

In [698]:
# perform wilcoxon signed-rank test on resources
from scipy.stats import wilcoxon
print(res_productivity)
wilcoxon(
    x=res_productivity['productivity_wo_boss'].to_numpy(),
    y=res_productivity['productivity_w_boss'].to_numpy(),
    alternative='greater'
)

direct_copresenting_boss   resource  mean_act_dur_wo_boss  \
3                         010-21-01              7.285714   
6                         010-23-02             21.339286   
9                         010-23-06             16.392045   
10                        010-23-07             20.272222   
13                        010-23-13             11.315789   
15                        010-24-01             25.894737   
16                        010-24-02             26.600000   
18                        010-24-04             17.818182   
19                        010-24-11              1.000000   
21                        010-24-13              3.428571   
23                        010-24-17              4.933333   
24                        010-24-23             15.555556   
26                        010-25-01             19.840000   
28                        010-25-06             14.222222   
29                        010-25-07             26.067797   
30                      

WilcoxonResult(statistic=131.0, pvalue=0.588536262512207)